In [7]:
%%writefile train_entropy_pruned_distilbert_advanced.py

import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
import random
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# -------------------
# Trainable Entropy Gate
# -------------------
class EntropyGate(nn.Module):
    def __init__(self, hidden_size=768, keep_ratio=0.75):
        super().__init__()
        self.keep_ratio = keep_ratio
        self.scorer = nn.Linear(hidden_size, 2)  # learnable
        self.budget_lambda = 0.1  # regularization strength

    def forward(self, hidden_states, attention_mask, labels=None):
        logits = self.scorer(hidden_states)          # [B, L, 2]
        probs = F.softmax(logits, dim=-1)            # [B, L, 2]
        ent = -(probs * probs.log()).sum(dim=-1)     # [B, L]
        k = max(1, int(self.keep_ratio * hidden_states.size(1)))
        topk_idx = ent.topk(k, dim=-1, largest=False).indices
        mask = torch.zeros_like(ent, dtype=torch.bool)
        mask.scatter_(1, topk_idx, True)
        pruned_states = hidden_states * mask.unsqueeze(-1)

        # Budget loss to encourage ~ρ fraction kept
        if labels is not None:
            avg_keep = mask.float().mean()
            budget_loss = self.budget_lambda * (avg_keep - self.keep_ratio) ** 2
        else:
            budget_loss = 0.0

        return pruned_states, attention_mask, budget_loss


# -------------------
# DistilBERT + Gate
# -------------------
from transformers.models.distilbert.modeling_distilbert import DistilBertModel

class DistilBertWithGate(nn.Module):
    def __init__(self, keep_ratio=0.5, num_labels=2):
        super().__init__()
        self.keep_ratio = keep_ratio
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.gate = EntropyGate(hidden_size=self.bert.config.dim, keep_ratio=keep_ratio)
        self.classifier = nn.Linear(self.bert.config.dim, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            output_hidden_states=True)
        hidden_states = outputs.hidden_states[1]  # after first block
        gated_states, _, budget_loss = self.gate(hidden_states, attention_mask, labels)
        outputs.last_hidden_state[:, :, :] = gated_states
        pooled = outputs.last_hidden_state[:, 0]
        logits = self.classifier(pooled)

        loss = None
        if labels is not None:
            ce_loss = F.cross_entropy(logits, labels)
            loss = ce_loss + budget_loss
        return {"loss": loss, "logits": logits}


# -------------------
# Metrics + Efficiency
# -------------------
def compute_metrics(preds, labels):
    acc = accuracy_score(labels, preds.argmax(-1))
    try:
        auc = roc_auc_score(labels, F.softmax(torch.tensor(preds), -1)[:, 1])
    except:
        auc = 0.0
    return {"acc": acc, "auc": auc}

def flops_proxy(seq_len, hidden_dim=768, rho=1.0, layers=12):
    """
    Analytical FLOPs estimate:
    FLOPs ~ layers * (L^2 * d) for attention.
    If pruning applies after 1 layer, adjust accordingly.
    """
    # here: first layer always full, later layers pruned
    return ( (seq_len**2) * hidden_dim ) + ( (layers-1) * ((rho*seq_len)**2) * hidden_dim )

# -------------------
# Training & Evaluation
# -------------------
def train(model, loader, device, epochs=2):
    optimizer = AdamW(model.parameters(), lr=5e-5)
    num_training_steps = len(loader) * epochs
    lr_scheduler = get_scheduler("linear", optimizer=optimizer,
                                 num_warmup_steps=0,
                                 num_training_steps=num_training_steps)
    model.train()
    for epoch in range(epochs):
        loop = tqdm(loader, desc=f"Epoch {epoch+1}")
        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            if "label" in batch:
                batch["labels"] = batch.pop("label")
            outputs = model(**batch)
            loss = outputs["loss"]
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            loop.set_postfix(loss=loss.item())


def evaluate(model, loader, device):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            if "label" in batch:
                batch["labels"] = batch.pop("label")
            outputs = model(**batch)
            logits = outputs["logits"].cpu()
            preds.append(logits)
            labels.append(batch["labels"].cpu())
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    return compute_metrics(preds, labels)


# -------------------
# Main
# -------------------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = load_dataset("glue", "sst2")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_fn(ex):
        return tokenizer(ex["sentence"], truncation=True,
                         padding="max_length", max_length=128)

    dataset = dataset.map(tokenize_fn, batched=True)
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    train_loader = DataLoader(dataset["train"], batch_size=16, shuffle=True)
    val_loader = DataLoader(dataset["validation"], batch_size=32)

    # baseline
    baseline = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=2
    ).to(device)
    train(baseline, train_loader, device, epochs=1)
    base_metrics = evaluate(baseline, val_loader, device)

    # proposed
    rho = 0.75
    pruned = DistilBertWithGate(keep_ratio=0.75, num_labels=2).to(device)
    train(pruned, train_loader, device, epochs=1)
    pruned_metrics = evaluate(pruned, val_loader, device)

    # FLOPs estimation
    seq_len = 128
    hidden_dim = 768
    layers = 12
    flops_base = flops_proxy(seq_len, hidden_dim, rho=1.0, layers=layers)
    flops_pruned = flops_proxy(seq_len, hidden_dim, rho=rho, layers=layers)
    flops_red = (1 - flops_pruned / flops_base) * 100

    print("\n==== Validation Results (SST-2) ====")
    print(f"Baseline: Acc={base_metrics['acc']:.4f}, AUC={base_metrics['auc']:.4f}, FLOPs={flops_base:.2e}")
    print(f"Proposed: Acc={pruned_metrics['acc']:.4f}, AUC={pruned_metrics['auc']:.4f}, "
          f"FLOPs={flops_pruned:.2e} ({flops_red:.1f}%↓)")


if __name__ == "__main__":
    main()


Overwriting train_entropy_pruned_distilbert_advanced.py


In [8]:
!python train_entropy_pruned_distilbert_advanced.py


2025-09-08 21:58:20.037590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757368700.057212   14558 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757368700.063147   14558 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757368700.078061   14558 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757368700.078084   14558 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757368700.078088   14558 computation_placer.cc:177] computation placer alr